In [ ]:
from utils.tf import *
from utils.db import *
from utils.gd import * 
from utils.clt_repetidos import *
from datetime import datetime
import cx_Oracle
import pandas as pd
import os
import plotly.express as px
import plotly.graph_objects as go
from utils.clt_repetidos import * 
from ydata_profiling import ProfileReport
import dask.dataframe as dd
import gc
import matplotlib.pyplot as plt

In [3]:
path = "/home/ale1726/proyects/datalake/clientes/data/productos/FFON"
path_exit = '/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados'

In [ ]:
date_now = datetime.now().strftime("%d_%m_%Y") 
path_exit =  os.path.join(path_exit,date_now)
os.makedirs(path_exit, exist_ok=True)

In [4]:
path = "/home/ale1726/proyects/datalake/clientes/data/productos/FFON/03_04_2025"

# Sistema tas

In [5]:
df_productos_tas = pd.read_csv(os.path.join(path, "movimientos_ffon_TAS.dat"), low_memory=False) 

In [6]:
df_productos_tas = pd.read_csv(os.path.join(path, "movimientos_ffon_TAS.dat"), low_memory=False) 
df_productos_tas["FVALOR_OPE_DT"] = pd.to_datetime(df_productos_tas["FVALOR_OPE_DT"])
df_productos_tas["FVENC_OPE_DT"] = pd.to_datetime(df_productos_tas["FVENC_OPE_DT"])
df_productos_tas['AÑO_FOP'] = df_productos_tas['FVALOR_OPE_DT'].dt.year
df_productos_tas['MES_FOP'] = df_productos_tas['FVALOR_OPE_DT'].dt.month

In [8]:
df_productos_tas.columns

Index(['CFE_M_OPERACION', 'CFE_C_BENEFICIARIO', 'FVALOR_OPE_DT',
       'FVENC_OPE_DT', 'MONTOENT_OPE_N', 'MONTOSAL_OPE_N', 'DESCRIPCION_OPE_S',
       'CONCEPTO', 'DEPARTAMENTO', 'ID_ORIGEN', 'BENEFICIARIO', 'CTA_INTERNA',
       'VIA', 'CALLE_BEN_S', 'NUM_BEN_S', 'COLONIA_BEN_S', 'MUNICIPIO_BEN_S',
       'TEL_BEN_S', 'CP_BEN_S', 'CONTACTO_BEN_S', 'NOMBRE_BEN_S', 'AP_BEN_S',
       'AM_BEN_S', 'RFC_BEN_S', 'PAIS_BEN_I', 'ENTIDAD_BEN_I', 'TIPO_BEN_I',
       'RAZONSOC_BEN_S', 'CE_BEN_S', 'STATUS_CE_BEN_N', 'PAIS', 'ENTIDAD',
       'CUENTA_BENS_2', 'CUENTA_BEN_S', 'BANCO', 'PLAZA', 'DIVISA'],
      dtype='object')

In [ ]:
agrupado_transacciones_anuales = (
    df_productos_tas.groupby(['AÑO_FOP'], as_index=False)
    .agg(
        num_transacciones=("CFE_M_OPERACION", 'count'),
        monto_x_año=("MONTOENT_OPE_N", 'sum'),
        mean_monto=("MONTOENT_OPE_N", 'mean'),
        median_monto=("MONTOENT_OPE_N", 'median'),
        mode_monto=("MONTOENT_OPE_N", lambda x: x.mode()[0] if not x.mode().empty else None),
        max_monto=("MONTOENT_OPE_N", 'max'),
        min_monto=("MONTOENT_OPE_N", 'min')
    )
    .sort_values('AÑO_FOP', ascending=True)
)


In [62]:
df_productos_tas["DURACION_OP"] = df_productos_tas["FVENC_OPE_DT"] - df_productos_tas["FVALOR_OPE_DT"]

In [ ]:
df_productos_tas

In [11]:
df_productos_tas.columns

Index(['CFE_M_OPERACION', 'CFE_C_BENEFICIARIO', 'FVALOR_OPE_DT',
       'FVENC_OPE_DT', 'MONTOENT_OPE_N', 'MONTOSAL_OPE_N', 'DESCRIPCION_OPE_S',
       'CONCEPTO', 'DEPARTAMENTO', 'ID_ORIGEN', 'BENEFICIARIO', 'CTA_INTERNA',
       'VIA', 'CALLE_BEN_S', 'NUM_BEN_S', 'COLONIA_BEN_S', 'MUNICIPIO_BEN_S',
       'TEL_BEN_S', 'CP_BEN_S', 'CONTACTO_BEN_S', 'NOMBRE_BEN_S', 'AP_BEN_S',
       'AM_BEN_S', 'RFC_BEN_S', 'PAIS_BEN_I', 'ENTIDAD_BEN_I', 'TIPO_BEN_I',
       'RAZONSOC_BEN_S', 'CE_BEN_S', 'STATUS_CE_BEN_N', 'PAIS', 'ENTIDAD',
       'CUENTA_BENS_2', 'CUENTA_BEN_S', 'BANCO', 'PLAZA', 'DIVISA', 'AÑO_FOP',
       'MES_FOP'],
      dtype='object')

In [60]:
df_productos_tas["PAIS_BEN_I"].value_counts()

PAIS_BEN_I
91095.0     426079
36335.0     189826
300.0       101101
151667.0     67310
310.0         5441
Name: count, dtype: int64

In [ ]:
df_productos_tas.groupby([]).agg(
    
    ).sort_values()

,CFE_M_OPERACION,CFE_C_BENEFICIARIO,FVALOR_OPE_DT,FVENC_OPE_DT,MONTOENT_OPE_N,MONTOSAL_OPE_N,DESCRIPCION_OPE_S,CONCEPTO,DEPARTAMENTO,ID_ORIGEN,...,STATUS_CE_BEN_N,PAIS,ENTIDAD,CUENTA_BENS_2,CUENTA_BEN_S,BANCO,PLAZA,DIVISA,AÑO_FOP,MES_FOP
0,118409949,26171,2024-12-18,2024-12-18 00:00:00,5.000000e+07,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,1,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12
1,118606655,26171,2024-12-23,2024-12-23 00:00:00,1.100000e+07,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,1,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12
2,118605621,26171,2024-12-23,2024-12-23 00:00:00,2.110000e+08,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,1,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12
3,118727193,26171,2024-12-26,2024-12-26 00:00:00,3.500000e+08,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,1,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12
4,118606702,26171,2024-12-23,2024-12-23 00:00:00,2.000000e+07,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,1,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064836,22314195,577872,2019-07-26,2019-07-26 00:00:00,2.720000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,1,.,.,NaN,1,Banco de Mexico,"MEXICO, DF",PESO MEXICANO,2019,7
2064837,22329781,577872,2019-07-26,2019-07-26 00:00:00,5.280000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,1,.,.,NaN,002180002362350178,"Banco Nacional de Mexico, S.A., Institucion de...","MEXICO, DF",PESO MEXICANO,2019,7
2064838,22329781,577872,2019-07-26,2019-07-26 00:00:00,5.280000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,1,.,.,NaN,1,Banco de Mexico,"MEXICO, DF",PESO MEXICANO,2019,7
2064839,22329782,577872,2019-07-26,2019-07-26 00:00:00,2.720000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,1,.,.,NaN,002180002362350178,"Banco Nacional de Mexico, S.A., Institucion de...","MEXICO, DF",PESO MEXICANO,2019,7


In [24]:
agrupado_transacciones_anuales = (
    df_productos_tas.groupby(['AÑO_FOP'], as_index=False)
    .agg(
        num_transacciones=("CFE_M_OPERACION", 'count'),
        monto_x_año=("MONTOENT_OPE_N", 'sum'),
        mean_monto=("MONTOENT_OPE_N", 'mean'),
        median_monto=("MONTOENT_OPE_N", 'median'),
        mode_monto=("MONTOENT_OPE_N", lambda x: x.mode()[0] if not x.mode().empty else None),
        max_monto=("MONTOENT_OPE_N", 'max'),
        min_monto=("MONTOENT_OPE_N", 'min')
    )
    .sort_values('AÑO_FOP', ascending=True)
)


In [29]:
agrupado_transacciones_anuales

,AÑO_FOP,num_transacciones,monto_x_año,mean_monto,median_monto,mode_monto,max_monto,min_monto
0,2019,379381,2.502027e+14,6.595024e+08,728778.260,0.0,5.067650e+11,0.0
1,2020,372590,2.886894e+14,7.748178e+08,1461473.120,0.0,1.003172e+12,0.0
2,2021,302708,2.117779e+14,6.996113e+08,1299588.890,0.0,2.978196e+11,0.0
3,2022,291738,2.124788e+14,7.283205e+08,1385416.670,0.0,8.305248e+10,0.0
4,2023,294764,2.098913e+14,7.120654e+08,2577824.025,0.0,2.000220e+11,0.0
5,2024,378150,2.907535e+14,7.688839e+08,1704828.205,0.0,9.499997e+10,0.0
6,2025,45510,3.853054e+13,8.466391e+08,885000.000,0.0,3.466125e+10,0.0


In [28]:
agrupado_transacciones_anuales.to_csv(os.path.join(path_exit, 'agrupado_transacciones_anuales.csv'),index=False)

In [25]:
agrupado_transacciones_mensual = (
    df_productos_tas.groupby(['AÑO_FOP',"MES_FOP"], as_index=False)
    .agg(
        num_transacciones=("CFE_M_OPERACION", 'count'),
        monto_x_año=("MONTOENT_OPE_N", 'sum'),
        mean_monto=("MONTOENT_OPE_N", 'mean'),
        median_monto=("MONTOENT_OPE_N", 'median'),
        mode_monto=("MONTOENT_OPE_N", lambda x: x.mode()[0] if not x.mode().empty else None),
        max_monto=("MONTOENT_OPE_N", 'max'),
        min_monto=("MONTOENT_OPE_N", 'min')
    )
    .sort_values(['AÑO_FOP','MES_FOP'], ascending=True)
)


In [26]:
agrupado_transacciones_mensual

,AÑO_FOP,MES_FOP,num_transacciones,monto_x_año,mean_monto,median_monto,mode_monto,max_monto,min_monto
0,2019,1,34820,2.070172e+13,5.945354e+08,1098802.500,0.0,5.067650e+11,0.0
1,2019,2,28432,1.655556e+13,5.822863e+08,850176.870,0.0,2.140000e+10,0.0
2,2019,3,30198,1.910364e+13,6.326129e+08,758655.195,0.0,2.165948e+10,0.0
3,2019,4,31294,2.051969e+13,6.557069e+08,697444.965,0.0,3.000000e+10,0.0
4,2019,5,34868,2.268578e+13,6.506190e+08,646722.220,0.0,3.222389e+10,0.0
...,...,...,...,...,...,...,...,...,...
69,2024,10,34208,2.451807e+13,7.167349e+08,1657144.990,0.0,2.891083e+10,0.0
70,2024,11,33765,2.826830e+13,8.372071e+08,1255000.000,0.0,2.862815e+10,0.0
71,2024,12,34191,2.951905e+13,8.633573e+08,1519130.200,0.0,3.281468e+10,0.0
72,2025,1,37767,3.187759e+13,8.440595e+08,885000.000,0.0,3.466125e+10,0.0


In [69]:
df_productos_tas

,CFE_M_OPERACION,CFE_C_BENEFICIARIO,FVALOR_OPE_DT,FVENC_OPE_DT,MONTOENT_OPE_N,MONTOSAL_OPE_N,DESCRIPCION_OPE_S,CONCEPTO,DEPARTAMENTO,ID_ORIGEN,...,PAIS,ENTIDAD,CUENTA_BENS_2,CUENTA_BEN_S,BANCO,PLAZA,DIVISA,AÑO_FOP,MES_FOP,DURACION_OP
0,118409949,26171,2024-12-18,2024-12-18,5.000000e+07,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12,0 days
1,118606655,26171,2024-12-23,2024-12-23,1.100000e+07,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12,0 days
2,118605621,26171,2024-12-23,2024-12-23,2.110000e+08,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12,0 days
3,118727193,26171,2024-12-26,2024-12-26,3.500000e+08,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12,0 days
4,118606702,26171,2024-12-23,2024-12-23,2.000000e+07,0.0,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060008,...,NaN,NaN,NaN,2647380203,"Banco Nacional de Comercio Exterior, S.N.C.","MEXICO, DF",PESO MEXICANO,2024,12,0 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064836,22314195,577872,2019-07-26,2019-07-26,2.720000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,.,.,NaN,1,Banco de Mexico,"MEXICO, DF",PESO MEXICANO,2019,7,0 days
2064837,22329781,577872,2019-07-26,2019-07-26,5.280000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,.,.,NaN,002180002362350178,"Banco Nacional de Mexico, S.A., Institucion de...","MEXICO, DF",PESO MEXICANO,2019,7,0 days
2064838,22329781,577872,2019-07-26,2019-07-26,5.280000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,.,.,NaN,1,Banco de Mexico,"MEXICO, DF",PESO MEXICANO,2019,7,0 days
2064839,22329782,577872,2019-07-26,2019-07-26,2.720000e+09,0.0,NaN,INICIO DE VENTA DIRECTO,DINERO,1057513,...,.,.,NaN,002180002362350178,"Banco Nacional de Mexico, S.A., Institucion de...","MEXICO, DF",PESO MEXICANO,2019,7,0 days


In [1]:
def get_full_name(first_name, last_name):
    full_name = first_name.title() + " " + last_name.title()
    return full_name


print(get_full_name("john", "doe"))

John Doe


In [ ]:
df_productos_tas['PAIS_BEN_I'].fillna('Desconocido')

In [70]:
agrupado_producto_año = (
    df_productos_tas.groupby(["AÑO_FOP"], as_index=False)
    .agg(
        cantidad_productos = ('CONCEPTO', "nunique"),
    )
).sort_values("AÑO_FOP", ascending=True)

In [71]:
agrupado_producto_año

,AÑO_FOP,cantidad_productos
0,2019,19
1,2020,19
2,2021,19
3,2022,19
4,2023,19
5,2024,19
6,2025,19


In [72]:
agrupado_producto_año = (
    df_productos_tas.groupby(["AÑO_FOP"], as_index=False)
    .agg(
        cantidad_productos = ('CONCEPTO', "unique"),
    )
).sort_values("AÑO_FOP", ascending=True)

In [73]:
agrupado_producto_año

,AÑO_FOP,cantidad_productos
0,2019,"[INICIO DE DEPOSITO RECIBIDO, VENCIMIENTO DE P..."
1,2020,"[INICIO DE DEPOSITO RECIBIDO, VENCIMIENTO DE P..."
2,2021,"[INICIO DE DEPOSITO RECIBIDO, VENCIMIENTO DE P..."
3,2022,"[INICIO DE DEPOSITO RECIBIDO, VENCIMIENTO DE P..."
4,2023,"[INICIO DE DEPOSITO RECIBIDO, VENCIMIENTO DE P..."
5,2024,"[INICIO DE DEPOSITO RECIBIDO, VENCIMIENTO DE P..."
6,2025,"[VENCIMIENTO DE PRESTAMO, INICIO DE PRESTAMO, ..."


In [24]:
agrupado_concepto = (
    df_productos_tas.groupby(["CONCEPTO"], as_index=False)
    .agg(
        num_transacciones = ('CFE_M_OPERACION', "count"),
        monto_avg = ('MONTOENT_OPE_N', "mean"),
        monto_sum = ('MONTOENT_OPE_N', "sum"),
        monto_max = ('MONTOENT_OPE_N', "max"),
        monto_min = ('MONTOENT_OPE_N', "min"),
        monto_median = ('MONTOENT_OPE_N', "median"),
    )
).sort_values("CONCEPTO", ascending=False)

In [25]:
agrupado_concepto

,CONCEPTO,num_transacciones,monto_avg,monto_sum,monto_max,monto_min,monto_median
18,VENCIMIENTO VENTA EN REPORTO,348637,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00
17,VENCIMIENTO DE PRESTAMO,3024,5.913210e+09,1.788155e+13,8.215895e+10,0.10,3.394555e+09
16,VENCIMIENTO DE INVERSION,12932,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00
15,VENCIMIENTO DE DEPOSITO,11751,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00
14,VENCIMIENTO DE COMPRA REPORTO,60919,2.060938e+09,1.255503e+14,1.480229e+10,97.65,2.000437e+09
13,RETIRO DE EFECTIVO,9682,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00
12,PAGO DE CUPON,5643,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00
11,PAGO DE CAPITAL,42773,0.000000e+00,0.000000e+00,0.000000e+00,0.00,0.000000e+00
10,INICIO DE VENTA REPORTO,539442,1.181612e+09,6.374112e+14,9.000000e+11,1.00,2.339062e+08
9,INICIO DE VENTA DIRECTO,176552,9.796537e+08,1.729598e+14,1.003172e+12,3.00,8.412793e+07


In [78]:
print(agrupado_concepto)

                         CONCEPTO  num_transacciones
18   VENCIMIENTO VENTA EN REPORTO             409226
17        VENCIMIENTO DE PRESTAMO               3032
16       VENCIMIENTO DE INVERSION              21965
15        VENCIMIENTO DE DEPOSITO              11825
14  VENCIMIENTO DE COMPRA REPORTO             108457
13             RETIRO DE EFECTIVO              16470
12                  PAGO DE CUPON              11510
11                PAGO DE CAPITAL             128285
10        INICIO DE VENTA REPORTO             638331
9         INICIO DE VENTA DIRECTO             221432
8              INICIO DE PRESTAMO               3030
7             INICIO DE INVERSION              44767
6     INICIO DE DEPOSITO RECIBIDO              11807
5        INICIO DE COMPRA REPORTO             176619
4     INICIO DE COMPRA EN DIRECTO             131508
3               INGRESO BXICOFTAS              19177
2              EGRESOS BXICO FTAS              19155
1                        DEPOSITO             

In [ ]:
agrupado_concepto_año = (
    df_productos_tas.groupby(["CONCEPTO","AÑO_FOP"], as_index=False)
    .agg(
        num_transacciones = ('CFE_M_OPERACION', "count"),
       
    )
).sort_values("AÑO_FOP", ascending=True)

In [75]:
agrupado_concepto_año

,CONCEPTO,AÑO_FOP,num_transacciones
0,COBRO DE CUPON,2019,10294
91,RETIRO DE EFECTIVO,2019,2848
35,INICIO DE COMPRA REPORTO,2019,37580
98,VENCIMIENTO DE COMPRA REPORTO,2019,23195
84,PAGO DE CUPON,2019,1940
...,...,...,...
97,RETIRO DE EFECTIVO,2025,306
34,INICIO DE COMPRA EN DIRECTO,2025,3683
27,INGRESO BXICOFTAS,2025,367
13,DEPOSITO,2025,209


# TODOS

In [9]:
path = "/home/ale1726/proyects/datalake/clientes/data/productos/FFON/03_04_2025"

In [10]:
path_exit = '/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados'

In [11]:
path_sistemas = [[sistema, sistema.split('_')[-1].split('.')[0]] for sistema in os.listdir(path) if 'movimientos' in sistema]

In [12]:
for sistema,name in path_sistemas:
    df_product = pd.read_csv(os.path.join(path, sistema), low_memory=False)
    df_product["FVALOR_OPE_DT"] = pd.to_datetime(df_product["FVALOR_OPE_DT"])
    df_product['AÑO_FOP'] = df_product['FVALOR_OPE_DT'].dt.year
    df_product['MES_FOP'] = df_product['FVALOR_OPE_DT'].dt.month
    

    
    agrupado_transacciones_anuales_monto_entrada = (
    df_product.groupby(['AÑO_FOP'], as_index=False)
    .agg(
        num_transacciones=("CFE_M_OPERACION", 'count'),
        monto_avg = ('MONTOENT_OPE_N', "mean"),
        monto_sum = ('MONTOENT_OPE_N', "sum"),
        monto_max = ('MONTOENT_OPE_N', "max"),
        monto_min = ('MONTOENT_OPE_N', "min"),
        monto_median = ('MONTOENT_OPE_N', "median"),
    )
    .sort_values('AÑO_FOP', ascending=True)
    )
    
    ruta1 = os.path.join(path_exit, f'agrupado_transacciones_anuales_monto_entrada_{name}.csv')
    print(ruta1)
    agrupado_transacciones_anuales_monto_entrada.to_csv(ruta1, index=False)
    
    
    agrupado_transacciones_anuales_monto_salida = (
    df_product.groupby(['AÑO_FOP'], as_index=False)
    .agg(
        num_transacciones=("CFE_M_OPERACION", 'count'),
        monto_avg = ('MONTOSAL_OPE_N', "mean"),
        monto_sum = ('MONTOSAL_OPE_N', "sum"),
        monto_max = ('MONTOSAL_OPE_N', "max"),
        monto_min = ('MONTOSAL_OPE_N', "min"),
        monto_median = ('MONTOSAL_OPE_N', "median"),
    )
    .sort_values('AÑO_FOP', ascending=True)
    )
    
    ruta1_1 = os.path.join(path_exit, f'agrupado_transacciones_anuales_monto_salida_{name}.csv')
    print(ruta1_1)
    agrupado_transacciones_anuales_monto_salida.to_csv(ruta1_1, index=False)
    
    agrupado_transacciones_mensual = (
    df_product.groupby(['AÑO_FOP',"MES_FOP"], as_index=False)
        .agg(
            num_transacciones=("CFE_M_OPERACION", 'count'),
        )
        .sort_values(['AÑO_FOP','MES_FOP'], ascending=True)
    )
    ruta2 = os.path.join(path_exit, f'agrupado_transacciones_mensual_{name}.csv')
    
    print(ruta2)
    agrupado_transacciones_mensual.to_csv(ruta2, index=False)

    
    agrupado_concepto_entrada = (
    df_product.groupby(["CONCEPTO"], as_index=False)
        .agg(
            num_transacciones = ('CFE_M_OPERACION', "count"),
            monto_avg = ('MONTOENT_OPE_N', "mean"),
            monto_sum = ('MONTOENT_OPE_N', "sum"),
            monto_max = ('MONTOENT_OPE_N', "max"),
            monto_min = ('MONTOENT_OPE_N', "min"),
            monto_median = ('MONTOENT_OPE_N', "median"),
        )
    ).sort_values("CONCEPTO", ascending=False)
    
    ruta3 = os.path.join(path_exit, f'agrupado_concepto_entrada_{name}.csv')
    print(ruta3)
    agrupado_concepto_entrada.to_csv(ruta3, index=False)
    
    agrupado_concepto_salida = (
    df_product.groupby(["CONCEPTO"], as_index=False)
        .agg(
            num_transacciones = ('CFE_M_OPERACION', "count"),
            monto_avg = ('MONTOSAL_OPE_N', "mean"),
            monto_sum = ('MONTOSAL_OPE_N', "sum"),
            monto_max = ('MONTOSAL_OPE_N', "max"),
            monto_min = ('MONTOSAL_OPE_N', "min"),
            monto_median = ('MONTOSAL_OPE_N', "median"),
        )
    ).sort_values("CONCEPTO", ascending=False)
    
    ruta4 = os.path.join(path_exit, f'agrupado_concepto_salida_{name}.csv')
    print(ruta4)
    agrupado_concepto_salida.to_csv(ruta4, index=False)
    
    agrupado_clientes_año = (
    df_product.groupby(["BENEFICIARIO","AÑO_FOP"], as_index=False)
    .agg(
        num_transacciones = ('CFE_M_OPERACION', "count"),
        monto_avg_entrada = ('MONTOENT_OPE_N', "mean"),
        monto_sum_entrada = ('MONTOENT_OPE_N', "sum"),
        monto_avg_salida = ('MONTOSAL_OPE_N', "mean"),
        monto_sum_salida = ('MONTOSAL_OPE_N', "sum")
    )
    ).sort_values("AÑO_FOP", ascending=False)
    
    ruta5 = os.path.join(path_exit, f'agrupado_clientes_año_{name}.csv')
    
    print(ruta5)
    agrupado_clientes_año.to_csv(ruta5, index=False)
    
    
    

/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_transacciones_anuales_monto_entrada_CUSTODIA.csv
/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_transacciones_anuales_monto_salida_CUSTODIA.csv
/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_transacciones_mensual_CUSTODIA.csv
/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_concepto_entrada_CUSTODIA.csv
/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_concepto_salida_CUSTODIA.csv
/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_clientes_año_CUSTODIA.csv
/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_transacciones_anuales_monto_entrada_FFON.csv
/home/ale1726/proyects/datalake/clientes/data/productos/FFON/agrupados/agrupado_transacciones_anuales_monto_salida_FFON.csv
/home/ale1726/proyects/datalake/clientes/data/producto

In [ ]:
agrupado_concepto = (
    df_productos_tas.groupby(["CONCEPTO"], as_index=False)
    .agg(
        num_transacciones = ('CFE_M_OPERACION', "count"),
        monto_avg = ('MONTOENT_OPE_N', "mean"),
        monto_sum = ('MONTOENT_OPE_N', "sum"),
        monto_max = ('MONTOENT_OPE_N', "max"),
        monto_min = ('MONTOENT_OPE_N', "min"),
        monto_median = ('MONTOENT_OPE_N', "median"),
    )
).sort_values("CONCEPTO", ascending=False)

In [7]:
agrupado_clientes_año = (
    df_productos_tas.groupby(["BENEFICIARIO","AÑO_FOP"], as_index=False)
    .agg(
        num_transacciones = ('CFE_M_OPERACION', "count"),
        monto_avg_entrada = ('MONTOENT_OPE_N', "mean"),
        monto_sum_entrada = ('MONTOENT_OPE_N', "sum"),
        monto_avg_salida = ('MONTOSAL_OPE_N', "mean"),
        monto_sum_salida = ('MONTOSAL_OPE_N', "sum"),
    )
).sort_values("BENEFICIARIO", ascending=False)

In [8]:
agrupado_clientes_año

,BENEFICIARIO,AÑO_FOP,num_transacciones,monto_avg_entrada,monto_sum_entrada,monto_avg_salida,monto_sum_salida
3406,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2023,3,1.016663e+06,3.049990e+06,0.000000e+00,0.000000e+00
3405,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2022,10,7.354178e+05,7.354178e+06,0.000000e+00,0.000000e+00
3404,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2021,7,1.801429e+06,1.261000e+07,0.000000e+00,0.000000e+00
3403,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2020,16,1.868125e+06,2.989000e+07,0.000000e+00,0.000000e+00
3402,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2019,2,1.245000e+06,2.490000e+06,0.000000e+00,0.000000e+00
...,...,...,...,...,...,...,...
2,".+TASA US, S.A. DE C.V., F.I.I.D.",2019,14,5.849925e+06,8.189896e+07,0.000000e+00,0.000000e+00
6,".+TASA US, S.A. DE C.V., F.I.I.D.",2024,4,9.625000e+06,3.850000e+07,0.000000e+00,0.000000e+00
3,".+TASA US, S.A. DE C.V., F.I.I.D.",2020,3,1.666667e+07,5.000000e+07,0.000000e+00,0.000000e+00
1,".+TASA PREVISIONAL, S.A. DE C.V., F.I.I.D.",2025,132,1.750491e+08,2.310648e+10,1.091376e+08,1.440616e+10


In [13]:
[
    {
        "Cliente": fil.BENEFICIARIO,
        "Año": "{:,.0f}".format(fil.AÑO_FOP),
        "Transacciones": "{:,.0f}".format(fil.num_transacciones),
        "Monto Entrada Promedio": "{:,.0f}".format(fil.monto_avg_entrada),
        "Monto Entrada Anual": "{:,.0f}".format(fil.monto_sum_entrada),
        "Monto Salida Promedio": "{:,.0f}".format(fil.monto_avg_salida),
        "Monto Salida Anual": "{:,.0f}".format(fil.monto_sum_salida),
    
    }
    for fil in agrupado_clientes_año.itertuples()
]

[{'Cliente': 'FINPO SA DE CV SOFOM ENR',
  'Año': '2,027',
  'Transacciones': '1',
  'Monto Entrada Promedio': '0',
  'Monto Entrada Anual': '0',
  'Monto Salida Promedio': '10',
  'Monto Salida Anual': '10'},
 {'Cliente': 'jaime enrique carrasco ortiz',
  'Año': '2,025',
  'Transacciones': '2',
  'Monto Entrada Promedio': '200,000',
  'Monto Entrada Anual': '400,000',
  'Monto Salida Promedio': '200,000',
  'Monto Salida Anual': '400,000'},
 {'Cliente': 'BANCOMEXT (CLABE)',
  'Año': '2,025',
  'Transacciones': '49',
  'Monto Entrada Promedio': '0',
  'Monto Entrada Anual': '0',
  'Monto Salida Promedio': '966,413',
  'Monto Salida Anual': '47,354,248'},
 {'Cliente': 'RODRIGO BALDEMAR SANTIAGO SANCHEZ',
  'Año': '2,025',
  'Transacciones': '1',
  'Monto Entrada Promedio': '0',
  'Monto Entrada Anual': '0',
  'Monto Salida Promedio': '490',
  'Monto Salida Anual': '490'},
 {'Cliente': 'BANCOMER BANCO',
  'Año': '2,025',
  'Transacciones': '167',
  'Monto Entrada Promedio': '841,959',
  

In [42]:
agrupado_clientes_año

,BENEFICIARIO,AÑO_FOP,num_transacciones,monto_avg,monto_sum,monto_max,monto_min,monto_median
3406,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2023,3,1.016663e+06,3.049990e+06,1.499990e+06,150010.00,1.399990e+06
3405,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2022,10,7.354178e+05,7.354178e+06,3.000000e+06,17503.75,2.133370e+05
3404,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2021,7,1.801429e+06,1.261000e+07,4.200000e+06,400000.00,9.000000e+05
3403,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2020,16,1.868125e+06,2.989000e+07,3.700000e+06,260000.00,1.400000e+06
3402,"ZURICH VIDA COMPA¿IA DE SEGUROS, S.A.",2019,2,1.245000e+06,2.490000e+06,1.900000e+06,590000.00,1.245000e+06
...,...,...,...,...,...,...,...,...
2,".+TASA US, S.A. DE C.V., F.I.I.D.",2019,14,5.849925e+06,8.189896e+07,1.800000e+07,1499990.00,3.999995e+06
6,".+TASA US, S.A. DE C.V., F.I.I.D.",2024,4,9.625000e+06,3.850000e+07,1.000000e+07,8500000.00,1.000000e+07
3,".+TASA US, S.A. DE C.V., F.I.I.D.",2020,3,1.666667e+07,5.000000e+07,2.000000e+07,10000000.00,2.000000e+07
1,".+TASA PREVISIONAL, S.A. DE C.V., F.I.I.D.",2025,132,1.750491e+08,2.310648e+10,7.000000e+08,0.00,1.999999e+08


In [43]:
df_productos_tas.columns

Index(['CFE_M_OPERACION', 'CFE_C_BENEFICIARIO', 'FVALOR_OPE_DT',
       'FVENC_OPE_DT', 'MONTOENT_OPE_N', 'MONTOSAL_OPE_N', 'DESCRIPCION_OPE_S',
       'CONCEPTO', 'DEPARTAMENTO', 'ID_ORIGEN', 'BENEFICIARIO', 'CTA_INTERNA',
       'VIA', 'CALLE_BEN_S', 'NUM_BEN_S', 'COLONIA_BEN_S', 'MUNICIPIO_BEN_S',
       'TEL_BEN_S', 'CP_BEN_S', 'CONTACTO_BEN_S', 'NOMBRE_BEN_S', 'AP_BEN_S',
       'AM_BEN_S', 'RFC_BEN_S', 'PAIS_BEN_I', 'ENTIDAD_BEN_I', 'TIPO_BEN_I',
       'RAZONSOC_BEN_S', 'CE_BEN_S', 'STATUS_CE_BEN_N', 'PAIS', 'ENTIDAD',
       'CUENTA_BEN_S', 'AÑO_FOP', 'MES_FOP'],
      dtype='object')

In [40]:
df_productos_tas

,CFE_M_OPERACION,CFE_C_BENEFICIARIO,FVALOR_OPE_DT,FVENC_OPE_DT,MONTOENT_OPE_N,MONTOSAL_OPE_N,DESCRIPCION_OPE_S,CONCEPTO,DEPARTAMENTO,ID_ORIGEN,...,ENTIDAD_BEN_I,TIPO_BEN_I,RAZONSOC_BEN_S,CE_BEN_S,STATUS_CE_BEN_N,PAIS,ENTIDAD,CUENTA_BEN_S,AÑO_FOP,MES_FOP
0,22358296,25437,2019-07-30,2019-07-30,0.000000e+00,6473898.69,NaN,RETIRO DE EFECTIVO,CAJA TAS,1054383,...,NaN,1,"BANCO UNION, S.A. EN LIQUIDACION (ACTUALMENT",NaN,1,NaN,NaN,NaN,2019,7
1,22933311,25526,2019-08-30,2019-08-30,1.700000e+08,0.00,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060003,...,NaN,1,BANJERCITO,NaN,1,NaN,NaN,NaN,2019,8
2,23548003,25526,2019-10-03,2019-10-03,1.000000e+09,0.00,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060003,...,NaN,1,BANJERCITO,NaN,1,NaN,NaN,NaN,2019,10
3,32831196,25526,2020-11-24,2020-11-24,5.000000e+07,0.00,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060003,...,NaN,1,BANJERCITO,NaN,1,NaN,NaN,NaN,2020,11
4,50518095,25526,2022-06-17,2022-06-17,4.000000e+09,0.00,NaN,INICIO DE DEPOSITO RECIBIDO,CALL TAS,1060003,...,NaN,1,BANJERCITO,NaN,1,NaN,NaN,NaN,2022,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1533970,19389099,38656,2019-01-08,2019-01-08,9.744000e+08,0.00,NaN,DEPOSITO,CAJA TAS,1022589,...,305820.0,1,SECRETARIA DE HACIENDA Y CRED PUB A TRAVES TES...,NaN,1,.,.,NaN,2019,1
1533971,19367810,38656,2019-01-07,2019-01-07,9.827700e+08,0.00,NaN,DEPOSITO,CAJA TAS,1022589,...,305820.0,1,SECRETARIA DE HACIENDA Y CRED PUB A TRAVES TES...,NaN,1,.,.,NaN,2019,1
1533972,19360249,38656,2019-01-04,2019-01-04,9.800050e+08,0.00,NaN,DEPOSITO,CAJA TAS,1022589,...,305820.0,1,SECRETARIA DE HACIENDA Y CRED PUB A TRAVES TES...,NaN,1,.,.,NaN,2019,1
1533973,19302132,38656,2019-01-02,2019-01-02,9.822450e+08,0.00,NaN,DEPOSITO,CAJA TAS,1022589,...,305820.0,1,SECRETARIA DE HACIENDA Y CRED PUB A TRAVES TES...,NaN,1,.,.,NaN,2019,1


In [8]:
df_product

,CFE_M_OPERACION,CFE_C_BENEFICIARIO,FVALOR_OPE_DT,FVENC_OPE_DT,MONTOENT_OPE_N,MONTOSAL_OPE_N,DESCRIPCION_OPE_S,CONCEPTO,DEPARTAMENTO,ID_ORIGEN,...,ENTIDAD_BEN_I,TIPO_BEN_I,RAZONSOC_BEN_S,CE_BEN_S,STATUS_CE_BEN_N,PAIS,ENTIDAD,CUENTA_BEN_S,AÑO_FOP,MES_FOP
0,46819417,14456,2022-03-11,2022-03-11 00:00:00,0.0,28069.29,NaN,NOMINA MATRIZ,PERSONAL,135095,...,3946.0,0,VELA MARTINEZ MARIA JUDITH,NaN,1,MEXICO,"MEXICO, D.F.",5015766.0,2022,3
1,19795468,14466,2019-02-06,2019-02-06 00:00:00,0.0,998449.47,47723,PAGO CON CHEQUE,PERSONAL,139206,...,3946.0,0,MORENO ARMENTA JUAN FRANCISCO,NaN,1,MEXICO,"MEXICO, D.F.",3610632.0,2019,2
2,19848106,14466,2019-02-13,2019-02-13 00:00:00,0.0,1247116.75,47902,PAGO CON CHEQUE,PERSONAL,139206,...,3946.0,0,MORENO ARMENTA JUAN FRANCISCO,NaN,1,MEXICO,"MEXICO, D.F.",3610632.0,2019,2
3,22805634,14392,2019-08-26,2019-08-26 00:00:00,0.0,30904.70,NaN,NOMINA MATRIZ,PERSONAL,76522,...,3946.0,0,GUERRERO ZENTENO JOSE ANTONIO,NaN,1,MEXICO,"MEXICO, D.F.",4863550.0,2019,8
4,27260493,14392,2020-04-02,2020-04-02 00:00:00,0.0,15107.82,NaN,NOMINA MATRIZ,PERSONAL,76522,...,3946.0,0,GUERRERO ZENTENO JOSE ANTONIO,NaN,1,MEXICO,"MEXICO, D.F.",4863550.0,2020,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111474,120597546,2725136,2025-01-24,2025-01-24 00:00:00,0.0,2000.00,-NAFIN- REEMBOLSO MEDICO,PAGO A PROVEEDOR,SERVICIO MEDICO,ROBC731008N78,...,302974.0,0,ROMERO BUGARIN CELENE,NaN,0,MEXICO,CIUDAD DE MEXICO,NaN,2025,1
111475,120573247,2727277,2025-01-27,2025-01-27 00:00:00,0.0,37654.00,50780,PAGO CON CHEQUE A PROV JURIDICO,OPERACION DEL GASTO,HAM5801015F9,...,34358.0,1,MUNICIPIO DE DURANGO,NaN,0,MEXICO,DURANGO,NaN,2025,1
111476,121607686,2744439,2025-02-06,2025-02-06 00:00:00,0.0,3256.95,-NAFIN- CAJA AHORRO,NOMINA MATRIZ,PERSONAL,21858-04,...,305820.0,0,NAVARRETE GONZALEZ MARIA DEL CARMEN,NaN,0,.,.,NaN,2025,2
111477,121607775,2744443,2025-02-06,2025-02-06 00:00:00,0.0,3256.95,-NAFIN- CAJA AHORRO,NOMINA MATRIZ,PERSONAL,21858-03,...,302974.0,0,NAVARRETE GONZALEZ MARIA DEL PILAR,NaN,0,MEXICO,CIUDAD DE MEXICO,NaN,2025,2


In [ ]:
agrupado_concepto